In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(r"C:\Users\mikoo\AI\osmo").load_data()
documents

Failed to load file C:\Users\mikoo\AI\osmo\cosmetics-07-00003-v2.pdf with error: RetryError[<Future at 0x2d02e4317f0 state=finished raised UnboundLocalError>]. Skipping...


[Document(id_='ada63541-a30f-404a-995c-a2a3f90efe59', embedding=None, metadata={'page_label': '1', 'file_name': 'A Principal Odor Map Unies Diverse Tasks in Human Olfactory Perception.pdf', 'file_path': 'C:\\Users\\mikoo\\AI\\osmo\\A Principal Odor Map Unies Diverse Tasks in Human Olfactory Perception.pdf', 'file_type': 'application/pdf', 'file_size': 3198464, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='A Principal Odor Map Uni\x00es Diverse Tasks in Human Olfactory Perception\nBrian K. Lee1†, Emily J. Mayhew2†, Benjamin Sanchez-Lengeling1, Jennifer N. Wei1, Wesley W.\

In [8]:
documents[0]

Document(id_='ada63541-a30f-404a-995c-a2a3f90efe59', embedding=None, metadata={'page_label': '1', 'file_name': 'A Principal Odor Map Unies Diverse Tasks in Human Olfactory Perception.pdf', 'file_path': 'C:\\Users\\mikoo\\AI\\osmo\\A Principal Odor Map Unies Diverse Tasks in Human Olfactory Perception.pdf', 'file_type': 'application/pdf', 'file_size': 3198464, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='A Principal Odor Map Uni\x00es Diverse Tasks in Human Olfactory Perception\nBrian K. Lee1†, Emily J. Mayhew2†, Benjamin Sanchez-Lengeling1, Jennifer N. Wei1, Wesley W.\n

In [ ]:
len(documents)

In [10]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

db = chromadb.PersistentClient(path="./OSMO")
chroma_collection = db.get_or_create_collection("first_batch")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
        # HuggingFaceEmbedding(model_name="OrlikB/KartonBERT-USE-base-v1"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(
    documents=documents,
    num_workers=4,  # dopasuj do liczby rdzeni CPU
    show_progress=True
)

In [11]:
from llama_index.core import VectorStoreIndex

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [12]:
import os
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.ollama import Ollama


llm = Ollama(model="qwen2:7b")
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
query_engine.query("Czym jest zakażenie szpitalne?")

c:\Users\mikoo\miniconda3\envs\primo\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Response(response='Zakaźenie szpitalne (hospital-acquired infection) refers to infections that patients acquire while they are in a hospital or other healthcare facility. This includes both infections that occur during their stay and those that develop soon after discharge. These infections can be caused by various microorganisms such as bacteria, viruses, fungi, or parasites. They pose significant challenges for healthcare systems due to the complexity of managing multiple co-existing health conditions and the potential complications they may cause.', source_nodes=[NodeWithScore(node=TextNode(id_='163a8e43-a080-4d1e-8d94-aad60bd9d777', embedding=None, metadata={'file_path': 'C:\\Users\\mikoo\\AI\\osmo\\BBBP.csv', 'file_name': 'BBBP.csv', 'file_type': 'text/csv', 'file_size': 148743, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], exclud

In [ ]:
# #### bez chromy
# from index import *
# from llama_index.core import Settings
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core.node_parser import SentenceSplitter

# Settings.llm = Ollama(model="qwen2:7b")
# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )
# documents = SimpleDirectoryReader(r"C:\Users\mikoo\Downloads\index\zakazenia\pdf").load_data()
# index = VectorStoreIndex.from_documents(documents, transformations=[SentenceSplitter()])
# query_engine = index.as_query_engine(response_mode="tree_summarize")

# query_engine.query("Czym jest zakażenie szpitalne?")

Response(response='Zakażenie szpitalne to każde zakażenie, które nastąpiło w związku z udzielaniem świadczeń zdrowotnych. To obowiązuje wtedy, gdy choroba nie pozostawała w momencie udzielania świadczeń zdrowotnych w okresie wylęgania albo wystąpiła po udzielaniu świadczeń zdrowotnych, w okresie nie dłuższym niż najdłuższy okres jej wylęgania.', source_nodes=[NodeWithScore(node=TextNode(id_='11804821-1602-4bcb-9caa-938bd113cd11', embedding=None, metadata={'page_label': '1', 'file_name': 'Załącznik nr 12 do P-7.5(ZH)-18 definicje.pdf', 'file_path': 'C:\\Users\\mikoo\\Downloads\\index\\zakazenia\\pdf\\Załącznik nr 12 do P-7.5(ZH)-18 definicje.pdf', 'file_type': 'application/pdf', 'file_size': 219359, 'creation_date': '2025-03-24', 'last_modified_date': '2025-03-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_mod

In [14]:
query_engine.query("74 chemically diverse mono-molecular odorants?")

Response(response='The document mentions a list of 74 chemically diverse mono-molecular odorants which includes various chemical compounds like drugs, chemicals, and other substances. The exact names of all these odorants are provided in the context information starting from "GR94839_E" through to "ICI199441". These represent a wide range of chemical classes including but not limited to esters, phenols, amines, etc.', source_nodes=[NodeWithScore(node=TextNode(id_='163a8e43-a080-4d1e-8d94-aad60bd9d777', embedding=None, metadata={'file_path': 'C:\\Users\\mikoo\\AI\\osmo\\BBBP.csv', 'file_name': 'BBBP.csv', 'file_type': 'text/csv', 'file_size': 148743, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships=

In [15]:
query_engine.query("""cross-efects between diferent odors6
, limiting the number of odor presentation per trial. Tis makes large-scale
approaches challenging. As a result, almost all available databases originate from trained expert panels or the
perfume industry, such as those from Dravnieks7
, Lefngwell8
, or Te Good Scent Company (dataset available
from https://github.com/pyrfume, where a well-curated collection of currently available databases is stored).
While such expert databases are a valuable resource for evaluations of olfactory quality and convince by high
reliability, they are somewhat limited in their validity and objectivity. Most of those use predefned labels (such
as “pleasant”, “sweet”, “bitter”, “musky”) """)

Response(response='In the context provided, there is mention of a phenomenon known as "competitive adsorption" between two odor components in an odor mixture. This implies that when different odors are present together, they may compete for interaction with the sensing material (like APTS-functionalized graphene). The document states that the distinct binding energies and charge transfer characteristics of these odors can lead to competitive adsorption, where one odor molecule has more chances to approach the functionalized graphene and dominate the signal characteristic in the mixture. This subsequently suppresses the signal from the other odor component.\n\nThis highlights an effect where different odors competing for interaction with a sensing element may influence each other\'s detection, making it challenging to accurately differentiate between them. The document further suggests that machine learning classifiers can help distinguish these odors by analyzing their unique sensing r

In [16]:
query_engine.query("""Overview of the study design and procedures. Te study has been conducted in Germany and in the
German language only, instructions and scales are presented in English here for better readability. First, all
participants flled in questionnaires using an online survey implemented via LimeSurvey. Ten, participants
were assigned to one out of nine odor sets, containing a subset of ten out of the total 74 mono-molecular odors.
Te odor samples were provided in small plastic containers. Two “anchor odors” were used in all odor sets
for comparison between groups, the other eight odors difered between groups. Te participants frst gave a
free description for each odor (in German) and then evaluated them again using eight visual analogue scales
and 16 further qualitative labels obtained from Keller & Vosshall""")

Response(response='The study overview indicates an experimental design aimed at exploring aspects of human olfactory perception. The research involved several stages:\n\n1. **Questionnaire Completion**: Participants filled in questionnaires using an online survey tool, LimeSurvey. This part likely covered various aspects related to the participants\' sensory experience and preferences.\n\n2. **Selection of Odor Sets**: Each participant was assigned to one out of nine different odor sets. These sets contained ten specific mono-molecular odors chosen from a total pool of 74 unique compounds. \n\n3. **Odor Presentation**: The selected odors were presented in small plastic containers to ensure consistent and controlled exposure.\n\n4. **Initial Descriptions**: Participants provided initial free-form descriptions of the odors (in German).\n\n5. **Reevaluation**: After providing their free descriptions, participants then reevaluated the odors using a combination of eight visual analogue scal